In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

> ### TorchText, TorchVision, TorchAudio는 데이터셋이 포함됨

In [ ]:
# Torchvision Dataset에는 두가지 argument가 포함
# transform, target_transform은 샘플 및 레이블을 각각 수정

training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

> ### 신경망을 정의하기 위해서는 nn.Module로부터 상속받은 클래스를 만들어야함.
> ### 신경망의 layer는 __init__에서 정의.
> ### 신경망이 어떻게 통과하는지는 forward()함수로 정의 

In [4]:
device = (
    "cuda" 
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device}")

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear(x)
        return logits

model = NN().to(device)

Using mps


In [ ]:
loss_fn = nn.CrossEntropyLoss()
# optimizer에는 어떤 모델의 파라미터를 조정할것인지 또한 argument로 들어가야함
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    

In [ ]:
# Saving Model
torch.save(model.state_dict(), "model.pth")

In [ ]:
# Loading Model
model = NN().to(device)
model.load_state_dict(torch.load("model.pth"))